# Electricity market basics

In this notebook, we will look at the basics of energy market operation. We will not yet consider energy networks, but instead look at an energy market located in a single point (so that we do not need to worry about any physical constraints on energy flows). NB: this notebook is heavy on computation, so it might take a while to load the examples.

This is not an electrical engineering course, so we will conveniently going to ignore very short-term operational problems (e.g., dynamics, and other things that happen in electrical systems on very short timescales of milliseconds to minutes); we will also ignore the intricacies of our modern alternating current networks. These are interesting but out of scope here. We will also confine ourselves to wholesale markets, where generators sell electricity to wholesale consumers (i.e., utilities and large industrial consumers). Utilities sell the power they buy on to consumers in consumer markets. These are less interesting from the perspective of energy market analysis, as consumers generally sign long-term contracts to buy electricity at a fixed price per unit, and competition depends more on marketing and service levels than on economics. You would expect, however, prices in consumer markets to be equal to a long-term average of wholesale market prices (as these represent the utility’s input costs), plus a markup to cover the utility’s other costs and profits, plus taxes. In the UK, consumer prices are approximately twice as high as wholesale market prices, but they do correlate over a number of years.

## Economic dispatch

The diagram below shows a simple example of four periods of energy market operation. Energy demand in the periods is assumed to be perfectly inelastic (i.e., not sensitive to prices), which is a reasonable approximation in the short term. Each period lasts 1 hour, such that, if a generator generates at 1MW in a particular period, it has generated 1MWh (i.e. MWs and MWhs are equivalent). Demand is given by

Period 1 1 MWh
Period 2 2 MWh
Period 3 3 MWh
Period 4 4 MWh

There are three types of generation. We have 2.1GW of coal-fired generation capacity, which has a constant marginal fuel cost of 8£/MWh and 2GW of gasfired generation a cost of 10£/MWh, while wind generation has no cost but a limited availability. Because of varying wind conditions, in each period, only the following percentage of its installed capacity is available:

Period 1 0 %
Period 2 0.5 %
Period 3 0.35 %
Period 4 0.9 %

You can choose the amount of installed wind capacity. In addition to their fuel costs, generators have to pay a carbon price if you select a nonzero value. Coal-fired generators emit 0.1t/MWh, and gas-fired generators 0.05t/MWh. Carbon prices are per tonne and due on all carbon the generators emit. Wind generators have no carbon emissions.

The widget below shows the optimal generation levels and prices which are obtained when we look for the intersection of the demand and supply curves in each period. This is also the perfectly competitive equilibrium, as we know that, without externalities or other distortions, perfect competition gives us the optimal quantities and prices. In mathematical terms, since there is no consumer surplus here (as the demand curve is vertical), we can obtain this point by minimising the total cost of generation (generation times variable generation costs, summed over all generators and all time periods), subject to a constraint that specifies that the sum of generation has to meet demand in each time period. Generation decisions are called dispatch decisions, so the generation levels correspond to optimal dispatch (or economic dispatch). The supply curve shows the merit order, i.e., the order in which generators should be dispatched in an optimal solution.

Experiment with the economic dispatch model below. Do you understand why prices are what they are (Note: they are the cost of the most expensive unit that is still producing, i.e. the price at the intersection of the supply and demand curves)? What happens to prices if we add progressively more wind into the system? Do you understand why? (this is called the merit order effect, because wind changes the supply curve). Would you be happy if you were one of the first to invest in wind turbines?

What happens if you increase carbon prices? Can you see coal and gas change place in the merit order? Do carbon emissions actually change if you increase the price? Why/why not?

If you had some energy storage capacity, when would you charge/discharge this? Would you be happy with more renewables or higher carbon prices, or not?

In [1]:
from IPython.display import IFrame
IFrame('https://eee-apps-836a08e1d2dd.herokuapp.com/electricity-market-basics-1', width=800, height=850)

## Carbon pricing

Spiel on carbon pricing.

In [5]:
from IPython.display import IFrame
IFrame('https://eee-apps-836a08e1d2dd.herokuapp.com/electricity-market-basics-2', width=800, height=550)

## Multiple time periods

apply above for multiple time periods...

In [8]:
from IPython.display import IFrame
IFrame('https://eee-apps-836a08e1d2dd.herokuapp.com/electricity-market-basics-3', width=800, height=1000)

## Energy storage

Energy storage and how it impacts here...

In [9]:
from IPython.display import IFrame
IFrame('https://eee-apps-836a08e1d2dd.herokuapp.com/electricity-market-basics-3', width=800, height=1000)

## Example for Great Britain

![GB](2016_generation_dispatch.png)

## Including technical constraints

The example above assumed that you could dispatch each generator at any level between 0 and its capacity. In reality, this is not the case. Conventional generators (coal, gas, etc.), in particular, generally have many constraints on their operations. Once important constraint is that they generally cannot increase or decrease their production very quickly - it takes time to make large adjustments to output, similarly to how a car cannot go from 0 to 200mph instantaneously. These constraints are called ramping constraints. They limit the amount of generation possible in a particular period to be within a given range of the generation in the previous period. Generators that are very flexible (e.g., newer gas-fired generators) can ramp quickly. Generators that are inflexible (e.g. especially older nuclear power plants) cannot ramp quickly.

In addition to ramping constraints, conventional generators generally have minimum run levels. They cannot produce a positive amount below these levels. A 1GW generator might, for instance have a minimum run level of 50%, which means that it can only either generate nothing, or generate between 0.5 and 1GW. Generators therefore do not only decide how much to generate, but also whether they will generate at all. To make this more complicated, it generally takes time to start up a generator, so if a generator is turned off, it may not be able to generate anything in the immediate future. Generators might also have minimum up and down times, such that when they turn off they have to remain off for a number of periods, and when they are on, they have to remain on for a number of time periods. Decisions about being on or off are called unit commitment decisions. They are binary - units are either on (“online”) or off (“offline”). Economic dispatch problems with unit commitment constraints are sometimes called unit commitment problems.

## Markets in the real world

The above models are representations of markets. In the real world, depending on the type of market (as we will see next week), there might be a market operator which on a regular (e.g., hourly) basis solves problems similar to those above, but with many more generators and time periods, and generally using bids that generators/consumers have declared instead of their actual costs and benefits. This market operator then instructs the generators and consumers what to do in every period, with penalties for deviating from these instruction. In other markets, such as the UK, there is no centralised dispatch - generators make their own decisions. However, if we assume perfect competition, we would get to the same solution, so large-scale versions of the
models above can still be used by industry or by researchers to predict what will happen.